In [1]:
%alias nbconvert nbconvert ./pygui.ipynb
%nbconvert

/Users/aaronciuffo/bin/develtools/nbconvert, ./pygui.ipynb,
[NbConvertApp] Converting notebook ./pygui.ipynb to python


In [2]:
%nbconvert 

/Users/aaronciuffo/bin/develtools/nbconvert, ./pygui.ipynb,
[NbConvertApp] Converting notebook ./pygui.ipynb to python


In [15]:
import PySimpleGUI as sg
import logging
import ArgConfigParse
import sys
import textwrap
import constants
from pathlib import Path

In [4]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# console handler for debug output
# ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# ch.setFormatter(formatter)

# logger.addHandler(ch)



In [5]:
def dict_to_keys(settings):
    '''convert INI keys into PySimpleGUI keys
    Args:
        settings(`dictionary`): single level dictionary {'key_one': 'value1', 'value_two': 'value 2'}
        
    Retruns:
        `dict: single level dictionary {'-KEY ONE-': 'value1', '-VALUE TWO-': 'value 2'}'''
    keys_to_elements = {}
    for key in settings:
        keys_to_elements[key] = '-'+key.replace('_', ' ').upper()+'-'
    
    return keys_to_elements

In [6]:
def keys_to_dict(settings):
    '''normalize PySimpleGUI keys back to INI friendly keys
    Args:
        settings(`dictionary`): single level dictionary {'-KEY ONE-': 'value1', '-VALUE TWO-': 'value 2'}
    
    Returns:
        `dict`: normalized INI frindly keys {'key_one': 'value1', 'value_two': 'value 2'}
    '''
    myDict = {}
    for key in settings:
        myDict[str(key).replace('-', '').replace(' ','_').lower()] = settings[key]
        
    return myDict

In [7]:
def write_back_config(updates, config, section):
    '''normalize PySimpleGUI variable keys and write back to argparser dictionary
    
       values that do not exist in the original configuration are ignored
    
    Args:
        updates(`dict`): dictionary of keys and values returned by PySG window
        config(`dict`): argparser nested INI style dictionary
        section(`str`): key for section in config dictionary
        
    Returns:
        tuple(dict, dict): (nested argparser style dictionary, ignored values)
        
    '''
    normalized_dict = keys_to_dict(updates)
    ignored = {}
    for key in normalized_dict:
        if key in config[section]:
            logging.debug(f'updating {section}[{key}]')
            config[section][key] = normalized_dict[key]
        else:
            logging.debug(f'ignoring key: {key}')
            ignored[key] = normalized_dict[key]
    
    return config, ignored

In [8]:
def create_settings_window(settings):
    '''basic settings window'''
    settings_keys_to_elements = dict_to_keys(settings)
   
    sg.theme('Material 2')
        
    def TextLabel(text): return sg.Text(text+':', justification='r', size=((20, 1)) )
        
    layout = [[sg.T('Settings', font='Any 18')],
              [TextLabel('Google Drive Path'), sg.Input(key='-GOOGLE DRIVE-'), sg.FolderBrowse(initial_folder='/Volumes/', target='-GOOGLE DRIVE-')],
              [sg.B('Save'), sg.B('Exit')]]

    window = sg.Window('Settings', layout, keep_on_top=True, finalize=True)
    
    for key in settings_keys_to_elements:
        try:
            window[settings_keys_to_elements[key]].update(value=settings[key])
        except Exception as e:
            logging.warn(f'problem updating GUI window from settings: key = {key}')
            
    
    return window

In [9]:
def create_adv_settings_window(settings):
    '''advanced settings window'''
    settings_keys_to_elements = dict_to_keys(settings)
    sg.theme('Material 2')
    
    def TextLabel(text): return sg.Text(text+':', justification='r', size=((20, 1)) )
        
    layout = [[sg.T('Advanced Settings', font='Any 18')],
              [TextLabel('Student Folders'), sg.Input(key='-GRADE LEVEL DIRS-'), sg.FileBrowse(target='-GRADE LEVEL DIRS-')],
              [TextLabel('Log Level'),  sg.Listbox(key='-LOG LEVEL-', values=('DEBUG', 'INFO', 'WARNING'), size=(20, 4))],
              [sg.B('Save'), sg.B('Exit')]]
    
    window = sg.Window('Advanced Settings', layout, keep_on_top=True, finalize=True)
    
    for key in settings_keys_to_elements:
        try:
            window[settings_keys_to_elements[key]].update(value=settings[key])
        except Exception as e:
            logging.warning(f'problem updating GUI window from settings: key = {key}, {e}')
            logging.warning(f'{type(e)}')
    return window

In [10]:
def create_main_window(settings):
    '''main window'''
    sg.theme('Material 2')
    
    layout = [[sg.T('Main Window')],
              [sg.T('Stuff here')],
#               [sg.ScrolledTextBox(size=(80,40), font='Courier 11')],
              [sg.B('Run'), sg.B('Exit'), sg.B('Settings'), sg.B('Advanced Settings')]]
    
    return sg.Window('Main Application', layout)

In [11]:
# def create_status_window():
#     sg.theme('Material 2')
    
#     layout = [[sg.Text('Print stuff')],
#                [sg.Output(size=(80, 20), key='-OUTPUT-')]]
    
#     return sg.Window('Status Window', layout)
    

In [12]:
def output(text, cols=65, indent=3):
    '''wrap and indent output to make it easier to read; print wrapped text
    Args:
        text(`str`): text to format
        cols(int): number of columns to wrap at
        indent(int): number of spaces to add to second-N lines
    Returns:
        `str`: wrapped and indended string with linebreaks'''
    wrapper = textwrap.TextWrapper()
    wrapper.width = cols
    wrapper.subsequent_indent = ' ' * indent
    text = wrapper.wrap(text)
    new_text = ''
    for i in text:
        new_text = new_text + i + '\n'
        
    print(new_text)
    return new_text

In [13]:
def main():
    
    logfile = constants.appName+'.log'
    logfile = Path('~/').expanduser()/Path(logfile)

#     # file handler for all levels
#     fh = logging.handlers.RotatingFileHandler(logfile, mode='a', maxBytes=2000, backupCount=2)
#     fh.setLevel(logging.DEBUG)
#     fh.setFormatter(formatter)

#     logger.addHandler(fh)
    
    
    parser = ArgConfigParse.ConfigFile(['./portfolioCreator.ini'])
    parser.parse_config()
    config = parser.config_dict
    
    
    logger.setLevel(config['advanced_settings']['log_level'])
    
    window = create_main_window(config['user_settings'])
    settings_updated = False
    
    sg.easy_print('Starting up', font='Courier 11', do_not_reroute_stdout=False, no_button=True, size=(80, 40), location=(10,10))
    
    logger.debug('')
    
    while True:
        if window is None:
            window = create_main_window(config['user_settings'])            
        
        
#         window.Refresh()

        event, values = window.read()
                
        if event in (sg.WIN_CLOSED, 'Exit'):
            break
            
        if event == 'Settings':
            event, values = create_settings_window(config['user_settings']).read(close=True)
            
            if event == 'Save':
                window.close()
                window = None
                
                config, ignored = write_back_config(values, config, 'user_settings')
                
                output('updating settings')
                ArgConfigParse.write(config, './portfolioCreator.ini')                
        
        if event == 'Advanced Settings':
            event, values = create_adv_settings_window(config['advanced_settings']).read(close=True)
            
            logger.debug(values)
            
            if event == 'Save':
                window.close()
                window = None
                
                # handle no changes to this setting
                if len(values['-LOG LEVEL-'])<1:
                    values['-LOG LEVEL-'] = [config['advanced_settings']['log_level']]
                
                # write back any changes
                config, ignored = write_back_config(values, config, 'advanced_settings')
                # store only the first item in the list
                if isinstance(config['advanced_settings']['log_level'], list):
                    config['advanced_settings']['log_level'] = config['advanced_settings']['log_level'][0]
                
                
                output('saving advanced settings')
                ArgConfigParse.write(config, './portfolioCreator.ini')
                
    


    sg.easy_print_close()
    window.close()
    

In [14]:
main()

DEBUG:__main__:
DEBUG:__main__:{'-GRADE LEVEL DIRS-': 'student_dirs.txt', 'Browse': '', '-LOG LEVEL-': ['DEBUG']}
DEBUG:__main__:{'-GRADE LEVEL DIRS-': 'student_dirs.txt', 'Browse': '', '-LOG LEVEL-': ['DEBUG']}
